# Introduction and Overview to Using LIGO/Virgo GW-strain data

These imports are more to check that you have the correct packages installed in your environment for use in this tutorial.

In [ ]:
%matplotlib inline
import gwpy
import gwosc
import pycbc
import lalinference
import h5py
import numpy as np

## Querying for event information

The gwosc.datasets module provides tools to search for datasets, including filtering on GPS times.

For example, we can search for what event datasets are available:

In [ ]:
from gwosc.datasets import find_datasets
events = find_datasets(type="event", detector="H1", segment=(1126051217, 1137254417))
print(events)

If we want to find the gps times for different science operations we can use the `run_segment` method.

In [ ]:
from gwosc.datasets import run_segment
print(run_segment('O1'))

Furthermore to access data from different events that are recorded we can find the urls where to download the data such for the first event.

In [ ]:
from gwosc.locate import get_event_urls
urls = get_event_urls('GW150914') # detector = , duration = )
print(urls)

To use this data we can download and read it in

In [ ]:
a = str(urls[0])
print(a)

In [ ]:
!wget "https://www.gw-osc.org/catalog/GWTC-1-confident/data/GW150914/H-H1_GWOSC_4KHZ_R1-1126259447-32.hdf5"

In [ ]:
from gwpy.timeseries import TimeSeries
hdata_down = TimeSeries.read('H-H1_GWOSC_4KHZ_R1-1126259447-32.hdf5', format='hdf5.losc')
type(hdata_down)

# Basic data querying exercises

- How long did O2 last, in human-readable format, i.e. (days, months, years)?
- How many events were detected during O1?
- What file URL contains data for V1 4096 seconds around GW170817?

# Strain data visualization

Now let's see what GW strain data looks like. We will again use the gwosc package to get the data.

In [ ]:
from gwosc.datasets import event_gps
gps = event_gps('GW170817')
print(gps)

In [ ]:
segment = (int(gps)-5, int(gps)+5)
print(segment)

In [ ]:
hdata = TimeSeries.fetch_open_data('H1', *segment, verbose=True)
print(hdata)

The TimeSeries class comes with a load of useful methods for doing basic operations with the GW-strain data, particularly plotting.

In [ ]:
plot = hdata.plot()

# Download and plot data

- Using the methods discussed in parts one and two, download the data from the Livingston detector for `GW150914` and plot the strain for a segement of 30 seconds around the merger.

# Frequency domain properties of the data

Now that we have some data, let's use the data from `GW150914` to begin interrogating the properties of the data

In [ ]:
# Set the data form the Livingston detect or ldata
ldata = 

For starters, let's take a basic fft of the data and see what we get.

In [ ]:
fft = ldata.fft()
print(fft)

We see that the above object is no longer an instance of our TimeSeries object that we originally read the data in as. It is now a FrequencySeries object, related, but with different properties and methods.

In [ ]:
plot = fft.abs().plot(xscale="log", yscale="log")
plot.show()

Although this is the amplitude spectral density, it doesn't look terribly much like the typical LIGO/Virgo like sensitivity plots that we see. This is because we need to window and bandpass the data. However, instead of doing all these steps separately, the TimeSeries and Frequency series objects have methods built-in to do this.

In [ ]:
asd = ldata.asd(fftlength=4, method="median")
plot = asd.plot()
ax = plot.gca()
ax.set_xlim(10, 1400)
ax.set_ylim(2e-24, 1e-20)
plot.show()

However, with such short data lengths, there is a lot of noise in the ASD. Given a longer set of data, can you plot the ASD around a different event?

# ASD Exercise

- Obtain data from a different event, let's say `GW170817` and plot the ASD for both the Hanford and Livingston detectors. Get a section of data longer than 120s.

The code below will take care of the plotting for you, please fill in teh steps to retrieve the data and perform the ASD.

In [ ]:
gps = 
# get Hanford data
hdata2 = 
hasd2 = 
# get Livingston data
ldata2 = 
lasd2 = 

In [ ]:
plot = hasd2.plot()
ax = plot.gca()
ax.set_xlim(10, 1400)
ax.set_ylim(5e-24, 1e-20)
# and plot using standard colours
ax.plot(lasd2, label='LIGO-Livingston', color='gwpy:ligo-livingston')
# update the Hanford line to use standard colour, and have a label
hline = ax.lines[0]
hline.set_color('gwpy:ligo-hanford')  # change colour of Hanford data
hline.set_label('LIGO-Hanford')

ax.set_ylabel(r'Strain noise [$1/\sqrt{\mathrm{Hz}}$]')
ax.legend()
plot.show()

# Q-tranforms and ASD spectrograms

Another, very interesting way to interrogate GW data is too look at the famous Q-transform plots which effectively show PSD evolution with time. Using the data we already have let's look at one of these 'spectrograms' which is a more basic form of the Q-plot.

In [ ]:
specgram = ldata.spectrogram2(fftlength=4, overlap=2, window='hann') ** (1/2.)
plot = specgram.plot()
ax = plot.gca()
ax.set_yscale('log')
ax.set_ylim(10, 1400)
ax.colorbar(
    clim=(1e-24, 1e-20),
    norm="log",
    label=r"Strain noise [$1/\sqrt{\mathrm{Hz}}$]",
)
plot.show()  # refresh

This looks fine, but it doesn't show the nice 'tracks' that became some famous with the first detection `GW150914` and `GW170817`. This requires a Q-transform plot. With getting into the details, a Q-transform plot essentially finds the optimal window size for each time which returns the highest Q value of that PSD. This allows us to pick out the highest power features as they sweep through frequency space while evolving with time.

In [ ]:
gps = event_gps('GW170817')
segment = (int(gps) - 30, int(gps) + 2)
hdata = TimeSeries.fetch_open_data('H1', *segment, verbose=True, cache=True)

Thankfully this q-transform is another hand method of the GWpy TimeSeries object, so all we need to do is:

In [ ]:
hq = hdata.q_transform(frange=(30, 500))
plot = hq.plot()
plot.colorbar(label="Normalised energy")

You can almost see a track in the data. However, we can optimze our choice of the range of allowed q to visualize this better.

In [ ]:
hq = hdata.q_transform(frange=(30, 500), qrange=(100, 110))
plot = hq.plot()
ax = plot.gca()
ax.set_epoch(gps)
ax.set_yscale('log')
ax.colorbar(label="Normalised energy")

# Q-transform exercise

- Can you repeat the above for `GW170817` but for the Livngston detector? What do you see?
- Can you modify the tranform or the visualization to better see the inspiral in the data?

# Generating GW signals

But how do we find a signal in the data? This is done via a process called matched filtering. To do matched filtering we will use the `PyCBC` package. Before we do any fancy searching for signals let's see some of the cool things we can do with `PyCBC`.

In [ ]:
from pycbc.waveform import get_td_waveform, td_approximants
import pylab

`PyCBC` has a myraid of different waveform templates that we can generate mock signals from. 

In [ ]:
# The options for waveforms in the timedomain are the following: 
print(td_approximants())

These represent many different approximants from theory and numerical simulations. With different effects captured by them. Whether PN approximation orders or tidal effects or numerically tuned waveforms, there is a lot of variety. The one we will use below is a very typical one when considering mergers of binary black holes.

In [ ]:
# The output of this function are the "plus" and "cross" polarizations of the gravitational-wave signal 
# as viewed from the line of sight at a given source inclination (assumed face-on if not provided)
hp, hc = get_td_waveform(approximant="SEOBNRv4_opt", # The waveform approximant
                         mass1=10, # Solar masses of the binary componenets
                         mass2=10, 
                         distance= 100, #Mpc
                         delta_t=1.0/4096, #4096 Hz sampling
                         f_lower=30) # Lower frequency cutoff to begin generating the waveform.
                                     # Effectively this sets the starting point of the timeseries.

pylab.plot(hp.sample_times, hp, label='Plus Polarization')
pylab.plot(hp.sample_times, hc, label='Cross Polarization')
pylab.xlabel('Time (s)')
pylab.legend()
pylab.grid()
pylab.show()

# Zoom in near the merger time#
pylab.plot(hp.sample_times, hp, label='Plus Polarization')
pylab.plot(hp.sample_times, hc, label='Cross Polarization')
pylab.xlabel('Time (s)')
pylab.xlim(-.01, .01)
pylab.legend()
pylab.grid()
pylab.show()

We can see our insprial now, with both polarizations, however, as we are not terribly sensitive to polarizations of GW signals yet with the current generation of detectors it is perfectly fine to select one polarization and use that as the complete wave.

An interesting thing to interrogate is how do these waveforms change with masses, distances, and inclinations of the binary system.

In [ ]:
max_amp = []
distances = np.arange(100,1000,10)
for d in distances:
    hp, hc = get_td_waveform(approximant="SEOBNRv4_opt",
                         mass1=10,
                         mass2=10,
                         delta_t=1.0/4096,
                         inclination = 0.0,
                         f_lower=30,
                         distance=d)
    max_amp.append(max(abs(hp)))

pylab.plot(distances,max_amp, label="GW-Strain")
pylab.plot(distances, 2.5e-19/distances, label="1/d fall-off")
pylab.xlabel('Distance [Mpc]')
pylab.legend()
pylab.ylabel('GW-strain maximum amp')
pylab.show()

# Waveform properties exercise

- Can you make a similar plot to the above for the variation of amplitude with inclination?
- "" with changing the mass of one of the binaries?

# Mock data with noise

Now let's generate some mock data with noise and perform mathced filtering on it.

In [ ]:
sample_rate = 4096 # samples per second
data_length = 1024 # seconds

# Generate a long stretch of white noise
white_noise = 1.0e-19*np.random.normal(size=[sample_rate * data_length])
times = np.arange(len(white_noise)) / float(sample_rate)

In [ ]:
hp1, _ = get_td_waveform(approximant="SEOBNRv4_opt",
                         mass1=10,
                         mass2=10,
                         delta_t=1.0/sample_rate,
                         f_lower=25)

pylab.figure()
pylab.title("The waveform hp1")
pylab.plot(hp1.sample_times, hp1)
pylab.xlabel('Time (s)')
pylab.ylabel('Amplitude')

waveform_start = numpy.random.randint(0, len(white_noise) - len(hp1))
white_noise[waveform_start:waveform_start+len(hp1)] += hp1.numpy()

pylab.figure()
pylab.title("Looks like random noise, right?")
pylab.plot(hp1.sample_times, white_noise[waveform_start:waveform_start+len(hp1)])
pylab.xlabel('Time (s)')
pylab.ylabel('Amplitude')

pylab.figure()
pylab.title("Signal in the data")
pylab.plot(hp1.sample_times, white_noise[waveform_start:waveform_start+len(hp1)])
pylab.plot(hp1.sample_times, hp1)
pylab.xlabel('Time (s)')
pylab.ylabel('Normalized amplitude')

Now let's repeat the process, but with colored noise from the expected noise spectrum on the `Hanford` detector in O3.

In [ ]:
import pycbc.noise
import pycbc.psd

In [ ]:
hp1, _ = get_td_waveform(approximant="SEOBNRv4_opt",
                         mass1=10,
                         mass2=10,
                         delta_t=1.0/sample_rate,
                         distance=100,
                         f_lower=25)

# The color of the noise matches a PSD which you provide
flow = 25.0
delta_f = 1.0 / 16
flen = int(sample_rate / delta_f) + 1
psd = pycbc.psd.aLIGOaLIGOO3LowT1800545(flen, delta_f, flow)

# Generate 32 seconds of noise at 4096 Hz
delta_t = 1.0 / 4096
tsamples = int(32 / delta_t)
ts = pycbc.noise.noise_from_psd(tsamples, delta_t, psd, seed=127)
colored_noise = ts

pylab.plot(ts.sample_times, colored_noise)
pylab.ylabel('Strain')
pylab.xlabel('Time (s)')
pylab.show()

waveform_start = numpy.random.randint(0, len(colored_noise) - len(hp1))
colored_noise[waveform_start:waveform_start+len(hp1)] += hp1.numpy()

pylab.figure()
pylab.title("Looks like random noise, right?")
pylab.plot(hp1.sample_times, colored_noise[waveform_start:waveform_start+len(hp1)])
pylab.xlabel('Time (s)')
pylab.ylabel('Normalized amplitude')

pylab.figure()
pylab.title("Signal in the data")
pylab.plot(hp1.sample_times, colored_noise[waveform_start:waveform_start+len(hp1)])
pylab.plot(hp1.sample_times, hp1)
pylab.xlabel('Time (s)')
pylab.ylabel('Normalized amplitude')

In fact if you see above we have added in the signal to the noise. Now let's perform matched filtering back on this signal to see if we can extract the waveform.

# Mock data generation exercises

Using the above:
- Can you generate a timeseries of colored noise (pick whatever PSD you want, tab complete psd.... and it will show many options) with a BNS inspiral in the data? For a BNS using 1.4 solar masses for each component and the waveform approximant `TaylorF2`. The other options are up to you, but I would recommend a starting frequency no higher than 30 Hz.

# Matched filtering

Using the BNS waveform generated in the above exercise and noisy data we can perform matched filtering in essentially one easy step.

In [ ]:
# Set BNS waveform as template, using plus polarization
template =
# Set the noisy data as the strain
det_strain = 
# Given the psd you used to generate the above noise data
det_psd =
# If your starting frequency was above 20 Hz set that as the cut freqnecy otherwise use 20Hz
f_cut =

In [ ]:
from pycbc.filter import matched_filter
# Now use this template to perform matched filtering against the simulated signal
snr = matched_filter(
    template, det_strain, psd=det_psd, low_frequency_cutoff=f_cut
)
snr.crop(15, 6)
peak = abs(snr).numpy().argmax()
snrp = snr[peak]
time = snr.sample_times[peak]
# The SNR is techincally complex but we use the abs_max as the report SNR of the signal
print("This signal is detected with an SNR of ", snrp)
print("This occurred at a time of ", time)

pylab.plot(det_strain.sample_times,snr)
pylab.xlabel('Times (s)')
pylab.ylabel('Matched Filtering SNR')

# Matched filtering on real data

Of course mathced filtering is normally nowhere this simple. We knew the signal, we had well-described colored gaussian noise, and by generation itself we did not need to bandpass or window the data. Let's finish by trying to perform matched filtering on a real dataset. The following example borrows heavily from the online tutorial of PyCBC matched filtering.

In [ ]:
from pycbc.catalog import Merger
from pycbc.filter import resample_to_delta_t, highpass

# As an example we use the GW150914 data
merger = Merger("GW150914")

# Get the data from the Hanford detector
strain = merger.strain('H1')

# Remove the low frequency content and downsample the data to 2048Hz
strain = highpass(strain, 15.0)
strain = resample_to_delta_t(strain, 1.0/2048)

pylab.plot(strain.sample_times, strain)
pylab.xlabel('Time (s)')
pylab.show()

We see that there are some weird edge effects happening. This is due to the finite size of our data set and the filter wraparound that occures when highpassing the data. Thus we must crop the ends of the data to remove this artifact.

In [ ]:
# Remove 2 seconds of data from both the beginning and end
conditioned = strain.crop(2, 2)

pylab.plot(conditioned.sample_times, conditioned)
pylab.xlabel('Time (s)')
pylab.show()

A fine place to start to estimate your noise is to use the predicted PSD's as we did above. However, in practice the noise does change from day to day, season to season, and even on approximately 15 minute timescale it can change somewhat. Thus we should estimate the PSD from the data set that have. As we are only concerned about the region over which the LIGO/Virgo data is calibrated [20 Hz, ~2000 Hz] we can estimate the PSD using Welch's method and taking many windows of the data over intervals much greater than our lower limiting frequency. 

In [ ]:
from pycbc.psd import interpolate, inverse_spectrum_truncation
# Estimate the power spectral density

# We use 4 second samples of our time series in Welch method.
psd = conditioned.psd(4)

# Now that we have the psd we need to interpolate it to match our data
# and then limit the filter length of 1 / PSD. After this, we can
# directly use this PSD to filter the data in a controlled manner
psd = interpolate(psd, conditioned.delta_f)

# 1/PSD will now act as a filter with an effective length of 4 seconds
# Since the data has been highpassed above 15 Hz, and will have low values
# below this we need to inform the function to not include frequencies
# below this frequency. 
psd = inverse_spectrum_truncation(psd, 4 * conditioned.sample_rate,
                                  low_frequency_cutoff=15)

Now we need to generate a template to use against the data, in practice a whole template bank will be used to search for many potential signals, but as we know that we are looking for BBH given this is `GW150914` we can use an approximant that is quite similar to the true signal.

In [ ]:
# We'll assume equal masses, and non-rotating black holes which is within the posterior probability
# of GW150914. 
m = 36 # Solar masses
hp, hc = get_td_waveform(approximant="SEOBNRv4_opt",
                     mass1=m,
                     mass2=m,
                     delta_t=conditioned.delta_t,
                     f_lower=20)

# We will resize the vector to match our data
hp.resize(len(conditioned))

# The waveform begins at the start of the vector, so if we want the
# SNR time series to correspond to the approximate merger location
# we need to shift the data so that the merger is approximately at the 
# first bin of the data.

# The cyclic_time_shift method shifts the timeseries by a given amount of time.
# It treats the data as if it were on a ring so points shifted off the end
# of the series reappear at the start. Note that time stamps are *not* in
# general affected (as the start time of the full array is shifted),
# but the index of each point in the vector is.
#
# By convention waveforms returned from `get_td_waveform` have their
# merger stamped with time zero, so we can use the start time to 
# shift the merger into position
pylab.figure()
pylab.title('Before shifting')
pylab.plot(hp.sample_times, hp)
pylab.xlabel('Time (s)')
pylab.ylabel('Strain')

template = hp.cyclic_time_shift(hp.start_time)

pylab.figure()
pylab.title('After shifting')
pylab.plot(template.sample_times, template)
pylab.xlabel('Time (s)')
pylab.ylabel('Strain')

In [ ]:
snr = matched_filter(template, conditioned,
                     psd=psd, low_frequency_cutoff=20)

# Remove time corrupted by the template filter and the psd filter
# We remove 4 seonds at the beginning and end for the PSD filtering
# And we remove 4 additional seconds at the beginning to account for
# the template length (this is somewhat generous for 
# so short a template). A longer signal such as from a BNS, would 
# require much more padding at the beginning of the vector.
snr = snr.crop(4 + 4, 4)

# Why are we taking an abs() here?
# The `matched_filter` function actually returns a 'complex' SNR.
# What that means is that the real portion correponds to the SNR
# associated with directly filtering the template with the data.
# The imaginary portion corresponds to filtering with a template that
# is 90 degrees out of phase. Since the phase of a signal may be 
# anything, we choose to maximize over the phase of the signal.
pylab.figure(figsize=[10, 4])
pylab.plot(snr.sample_times, abs(snr))
pylab.ylabel('Signal-to-noise')
pylab.xlabel('Time (s)')
pylab.show()

peak = abs(snr).numpy().argmax()
snrp = snr[peak]
time = snr.sample_times[peak]

print("We found a signal at {}s with SNR {}".format(time, 
                                                    abs(snrp)))

# Matched filtering exercise

- Can you repeat the above process to find a merger signal for a different event? 

- As a challenge instead of getting the data directly from the Merger or Event methods, can you use everything we have learned thus far, download the data for an arbitrary segment size of data around your chosen event, read it in, plot the data, then proceed with the matched filtering? If you downloading an `hdf5` and reading in as described above it will be useful to use the `GWpy` method `to_pycbc` method to convert the data object into the class instance needed for the matched filtering example above. Best of luck!!